In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
import warnings
warnings.filterwarnings(action='ignore')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
book_path='/content/gdrive/MyDrive/SoftwareProject2/sources/Bookshop(3).xlsx'

In [ ]:
import pandas as pd
all_sheets = pd.read_excel(book_path, sheet_name=None)


In [ ]:
sales_data = all_sheets['Sales']

# Convert 'Sale Date' to datetime
sales_data['Sale Date'] = pd.to_datetime(sales_data['Sale Date'])

# Set 'Sale Date' as the index
sales_data.set_index('Sale Date', inplace=True)

# Group by ISBN and month, and count the number of sales
monthly_sales = sales_data.groupby([sales_data.index.to_period('M'), 'ISBN']).size().reset_index(name='Sales')

# Pivot the table to have dates as rows and books as columns, filling missing values with 0
sales_pivot = monthly_sales.pivot_table(index='Sale Date', columns='ISBN', values='Sales', fill_value=0)


In [ ]:
# Split the data into training and test sets
train_data = sales_pivot[np.logical_and(sales_pivot.index.year < 2193,np.logical_not(np.logical_and(sales_pivot.index.month==12, sales_pivot.index.year==2192)))]
test_data = sales_pivot[np.logical_or(np.logical_and(sales_pivot.index.month==12, sales_pivot.index.year==2192),sales_pivot.index.year==2193)]


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
import numpy as np

# Function to perform the ADF test to check for stationarity
def adf_test(timeseries):
    result = adfuller(timeseries, autolag='AIC')
    return result[1]  # p-value

# Function to find the optimal SARIMA order
def optimize_sarima(parameters_list, seasonal_parameters_list, d, D, s, time_series):
    """
    Return dataframe with parameters and corresponding AIC

    parameters_list - list with (p, q, P, Q) tuples
    d, D - integration order, seasonal integration order
    s - length of season
    time_series - the observed time series
    """

    results = []

    for param in parameters_list:
        for param_seasonal in seasonal_parameters_list:
            try:
                model = SARIMAX(time_series,
                                order=(param[0], d, param[1]),
                                seasonal_order=(param_seasonal[0], D, param_seasonal[1], s),
                                enforce_stationarity=False,
                                enforce_invertibility=False,
                                disp=False)
                results.append([param, param_seasonal, model.fit(disp=False).aic])
            except:
                continue

    result_df = pd.DataFrame(results, columns=['param', 'param_seasonal', 'aic'])
    result_df = result_df.sort_values(by='aic', ascending=True).reset_index(drop=True)

    return result_df

# Select the sales data for a single ISBN
# We choose the first ISBN from the list as a demonstration
isbn_to_forecast = sales_pivot.columns[0]
single_book_sales = train_data[isbn_to_forecast]

# Check if the time series is stationary
adf_p_value = adf_test(single_book_sales)

# If the p-value is greater than 0.05, we conclude that the time series is not stationary
# and needs differencing
if adf_p_value > 0.05:
    d = 1
else:
    d = 0

# Set seasonal parameters
# Assuming a yearly seasonality for book sales
s = 12
D = 1 if d == 0 else 0  # If we already have differenced, no need for seasonal differencing

# Define p, q, P, Q ranges to test
p = range(0, 3)  # AR terms
q = range(0, 3)  # MA terms
P = range(0, 2)  # Seasonal AR terms
Q = range(0, 2)  # Seasonal MA terms

# Create a list of all possible combinations of p, q, P, and Q
parameters = [(x[0], x[1]) for x in list(np.ndindex((len(p), len(q))))]
seasonal_parameters = [(x[0], x[1]) for x in list(np.ndindex((len(P), len(Q))))]

# Optimize SARIMA parameters
optimal_params_df = optimize_sarima(parameters, seasonal_parameters, d, D, s, single_book_sales)

optimal_params_df.head()


,param,param_seasonal,aic
0,"(0, 2)","(1, 1)",323.220844
1,"(2, 2)","(1, 1)",323.794066
2,"(1, 2)","(1, 1)",323.909638
3,"(2, 1)","(1, 1)",328.716175
4,"(2, 0)","(1, 1)",332.317311


In [ ]:
min_value_row = optimal_params_df.loc[optimal_params_df['aic'].idxmin()]
optimal_p, optimal_q = min_value_row['param']
optimal_P, optimal_Q = min_value_row['param_seasonal']

# Fit the SARIMA model with the optimal parameters
best_sarima_model = SARIMAX(single_book_sales,
                            order=(optimal_p, d, optimal_q),
                            seasonal_order=(optimal_P, D, optimal_Q, s),
                            enforce_stationarity=False,
                            enforce_invertibility=False)
best_sarima_model_fit = best_sarima_model.fit(disp=False)
best_sarima_model_fit.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                    989-28-229-0197-6   No. Observations:                   59
Model:             SARIMAX(0, 1, 2)x(1, 0, [1], 12)   Log Likelihood                -156.610
Date:                              Sat, 04 Nov 2023   AIC                            323.221
Time:                                      16:58:15   BIC                            332.027
Sample:                                  01-31-2188   HQIC                           326.468
                                       - 11-30-2192                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.4736      0.289     -1.641      0.101      -1.039       0.092
ma.L2         -0.3234      0.178     -1.819      0.069      -0.672       0.025
ar.S.L12       0.9896      0.132      7.508      0.000       0.731       1.248
ma.S.L12      -1.0000   1.17e+04  -8.52e-05      1.000    -2.3e+04     2.3e+04
sigma2        55.4071    6.5e+05   8.52e-05      1.000   -1.27e+06    1.27e+06
===================================================================================
Ljung-Box (L1) (Q):                   0.17   Jarque-Bera (JB):                48.08
Prob(Q):                              0.68   Prob(JB):                         0.00
Heteroskedasticity (H):               2.96   Skew:                             1.24
Prob(H) (two-sided):                  0.05   Kurtosis:                         7.55
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
single_book_sales_test = test_data[isbn_to_forecast]
single_book_sales_test

Sale Date
2192-12    38
2193-01    18
2193-02    16
2193-03    26
2193-04    38
2193-05    34
2193-06    43
2193-07    83
2193-08    77
2193-09    46
2193-10    38
2193-11    34
2193-12    49
Freq: M, Name: 989-28-229-0197-6, dtype: int64

In [ ]:
# Predictions for 2194
from sklearn.metrics import mean_squared_error
sarima_predictions = best_sarima_model_fit.forecast(len(single_book_sales_test))
sarima_mse = mean_squared_error(single_book_sales_test, sarima_predictions)

new_sarima_residuals = single_book_sales_test - sarima_predictions
new_sarima_residuals, sarima_predictions, sarima_mse


(Sale Date
 2192-12    -2.919839
 2193-01   -12.540945
 2193-02    -8.134614
 2193-03    -5.451723
 2193-04    -6.965524
 2193-05    -9.078283
 2193-06     0.086137
 2193-07    21.810217
 2193-08    14.016519
 2193-09   -10.086040
 2193-10    -2.540977
 2193-11    -4.988868
 2193-12     5.182994
 Freq: M, dtype: float64,
 2192-12    40.919839
 2193-01    30.540945
 2193-02    24.134614
 2193-03    31.451723
 2193-04    44.965524
 2193-05    43.078283
 2193-06    42.913863
 2193-07    61.189783
 2193-08    62.983481
 2193-09    56.086040
 2193-10    40.540977
 2193-11    38.988868
 2193-12    43.817006
 Freq: M, Name: predicted_mean, dtype: float64,
 94.20926830035943)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

# Assuming 'new_sarima_residuals' contains the residuals from your SARIMA model
# Reshape the residuals data for LSTM model
residuals_data = new_sarima_residuals.values.reshape(-1, 1)

# Normalize the residuals data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_residuals = scaler.fit_transform(residuals_data)

# Specify LSTM training parameters
n_input = 1  # number of inputs (we will use one lagged observation)
n_features = 1  # number of features (this dataset is univariate)
batch_size = 1  # number of observations per batch
epochs = 200  # number of epochs

# Generate training data for LSTM
generator = TimeseriesGenerator(scaled_residuals, scaled_residuals, length=n_input, batch_size=batch_size)

In [ ]:
# Define LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(n_input, n_features), return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(50, activation='relu', return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(25, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_25 (LSTM)              (None, 1, 50)             10400     
                                                                 
 dropout_24 (Dropout)        (None, 1, 50)             0         
                                                                 
 lstm_26 (LSTM)              (None, 50)                20200     
                                                                 
 dropout_25 (Dropout)        (None, 50)                0         
                                                                 
 dense_25 (Dense)            (None, 25)                1275      
                                                                 
 dropout_26 (Dropout)        (None, 25)                0         
                                                                 
 dense_26 (Dense)            (None, 1)               

In [ ]:
# Fit the LSTM model
lstm_model.fit(generator, epochs=epochs, verbose=0)

# Make predictions
lstm_predictions_scaled = lstm_model.predict(generator)

# Inverse transform to get the actual predictions
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)

12/12 [==============================] - 0s 2ms/step


In [ ]:
mean_squared_error(single_book_sales_test.to_numpy()[1:], sarima_predictions.to_numpy()[1:] + lstm_predictions.ravel())

86.18941915592457

In [ ]:
lstm_predictions

array([[-1.8415096],
       [-4.281099 ],
       [-3.329308 ],
       [-2.5911107],
       [-3.020167 ],
       [-3.559148 ],
       [-0.9119589],
       [ 7.243624 ],
       [ 4.005067 ],
       [-3.7998712],
       [-1.7267336],
       [-2.456479 ]], dtype=float32)

In [ ]:
import math

final_predictions = np.array([math.floor(x) for x in sarima_predictions.to_numpy()[1:] + lstm_predictions.ravel()])
final_predictions

array([28, 19, 28, 42, 40, 39, 60, 70, 60, 36, 37, 41])